In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier 

In [7]:
os.getcwd()

'/Users/coco/Downloads'

In [8]:
os.chdir('/Users/coco/Downloads')

In [9]:
train = pd.read_csv('train.csv')

In [10]:
train.head(15)

,VAR1,VAR2,VAR3,clean_date,purchase_act,purchaser_ind,first_purchase_date
0,00002acbe042d69,2013-07-11,EmailOpen,20130711,0,NaN,NaN
1,00002acbe042d69,2013-07-11,FormSubmit,20130711,0,NaN,NaN
2,00002acbe042d69,2013-07-15,EmailOpen,20130715,0,NaN,NaN
3,00002acbe042d69,2013-07-17,EmailOpen,20130717,0,NaN,NaN
4,00002acbe042d69,2013-07-18,EmailOpen,20130718,0,NaN,NaN
5,00002acbe042d69,2013-07-25,EmailOpen,20130725,0,NaN,NaN
6,00002acbe042d69,2013-07-26,EmailOpen,20130726,0,NaN,NaN
7,00002acbe042d69,2013-08-01,EmailOpen,20130801,0,NaN,NaN
8,00002acbe042d69,2013-08-05,EmailOpen,20130805,0,NaN,NaN
9,00002acbe042d69,2013-08-12,EmailOpen,20130812,0,NaN,NaN


In [11]:
train.columns = ['UserId','Purchase_Date','Event','Clean_Date','Purchase_Cnt','Y','First_Purchase_Date']

In [12]:
#Replace NA Values with 0
train['Y'] = train['Y'].fillna(0).astype(int)
train['First_Purchase_Date'] = train ['First_Purchase_Date'].fillna(0).astype(int)


In [13]:
GroupKey='UserId'
Date = 'Purchase_Date'
Event = 'Event'
Event_Steps = ['EmailOpen','EmailClickthrough','FormSubmit','WebVist','PageView','Purchase']



In [14]:
def funnelize (group, Event_Steps):
    curr = 0
    cnts = [0 for s in Event_Steps]
 
    for i,row in group.iterrows():
        event = row[Event]
        if event in Event_Steps:
            idx = Event_Steps.index(event)
            cnts[idx] +=1
    return cnts 

In [15]:
train.sort_values(by=Date,ascending=True, inplace=True)

In [16]:
Event_Cnts = train.groupby(GroupKey).apply(funnelize,Event_Steps)
        
        

In [17]:
NewTrain = pd.DataFrame(list(Event_Cnts.values),index = Event_Cnts.index,columns = Event_Steps)

In [18]:
NewTrain.head(10)

,EmailOpen,EmailClickthrough,FormSubmit,WebVist,PageView,Purchase
UserId,,,,,,
00002acbe042d69,9,0,1,0,0,0
0000a3a34bd07e6,20,0,0,0,0,0
0000ecf58c94668,11,0,1,0,0,6
00012205dd4613b,2,0,0,0,0,0
000198d39a568cf,0,0,0,0,0,1
0001b315b991cf0,1,0,0,0,0,0
00022618457fcf2,0,0,0,0,0,1
000256826de4432,0,0,0,0,0,1
000269150971d08,3,0,0,0,0,0


In [19]:
NewTrain['PurchaseBehavior'] = np.where(NewTrain['Purchase']>=1,1, 0)

In [20]:
Train = NewTrain.sample(frac=0.7)

In [21]:
Test = NewTrain.loc[~NewTrain.index.isin(Train.index)]

In [22]:
Train = Train.as_matrix()
Test = Test.as_matrix()

In [23]:
XTr = Train[:,:-2]

In [24]:
YTr = Train[:,-1]

In [25]:
XTe = Test[:,:-2]
YTe = Test[:,-1]

In [26]:
model = LogisticRegression()

In [27]:
model.fit(XTr, YTr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
print(model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [29]:
predicted = model.predict(XTe)

In [30]:
expected = YTe

In [31]:
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       0.70      1.00      0.82     71270
          1       0.75      0.02      0.04     31200

avg / total       0.71      0.70      0.59    102470

[[71026   244]
 [30478   722]]


In [32]:
expected.shape

(102470,)

In [33]:
predicted.shape

(102470,)

In [34]:
print predicted 

[0 0 0 ..., 0 0 0]


In [35]:
sum(predicted)

966

In [36]:
forest = RandomForestClassifier(n_estimators = 100)

In [37]:
XTr.shape

(239098, 5)

In [38]:
forest = forest.fit(XTr,YTr)

In [39]:
output = forest.predict(XTe)

In [40]:
output 

array([0, 0, 1, ..., 1, 0, 0])

In [41]:
pd.crosstab(YTe, output, rownames=['actual'], colnames=['preds'])

preds,0,1
actual,,
0,69252,2018
1,9901,21299


In [42]:
importances = forest.feature_importances_

In [43]:
importances 

array([ 0.72675852,  0.        ,  0.2119421 ,  0.        ,  0.06129938])

In [44]:
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

Feature ranking:


In [45]:

for f in range(XTr.shape[1]):
    print("%d. feature %d (%f)" % (f +1 , indices[f], importances[indices[f]]))

1. feature 0 (0.726759)
2. feature 2 (0.211942)
3. feature 4 (0.061299)
4. feature 3 (0.000000)
5. feature 1 (0.000000)


In [46]:
NewTesting = pd.read_csv('test.csv')

In [47]:
NewTesting.head()

,VAR1,VAR2,VAR3,clean_date
0,0000ecf58c94668,2015-05-20,EmailOpen,20150520
1,0000ecf58c94668,2015-06-11,EmailOpen,20150611
2,0000ecf58c94668,2015-06-29,EmailOpen,20150629
3,0000ecf58c94668,2015-07-05,EmailOpen,20150705
4,0000ecf58c94668,2015-07-29,EmailOpen,20150729


In [48]:
NewTesting.columns = ['UserId','Purchase_Date','Event','Clean_Date']

In [49]:
GroupKey='UserId'
Date = 'Purchase_Date'
Event = 'Event'
Event_Steps = ['EmailOpen','EmailClickthrough','FormSubmit','WebVist','PageView']

In [50]:
def funnelize (group, Event_Steps):
    curr = 0
    cnts = [0 for s in Event_Steps]
 
    for i,row in group.iterrows():
        event = row[Event]
        if event in Event_Steps:
            idx = Event_Steps.index(event)
            cnts[idx] +=1
    return cnts 

In [51]:
NewTesting.sort_values(by=Date,ascending=True, inplace=True)

In [52]:
Event_Cnts = NewTesting.groupby(GroupKey).apply(funnelize,Event_Steps)

In [53]:
Testing = pd.DataFrame(list(Event_Cnts.values),index = Event_Cnts.index,columns = Event_Steps)

In [54]:
Testing.head(10)

,EmailOpen,EmailClickthrough,FormSubmit,WebVist,PageView
UserId,,,,,
0000ecf58c94668,5,0,0,0,0
000132f21724ee2,1,1,0,0,0
0001b315b991cf0,3,0,0,0,0
000269150971d08,4,0,0,0,0
000297ba4fa5026,3,0,0,0,0
000342505b41079,7,0,0,0,0
0003b304dd7cfae,1,0,0,0,0
0003ee709c3b5dc,3,0,0,0,0
00049247bdf8ddf,2,1,1,0,1


In [55]:
Test = Testing.as_matrix()

In [70]:
Test.shape

(132810, 5)

In [71]:
output = forest.predict(Test)

In [72]:
output = forest.predict_proba(Test)

In [73]:
output = pd.DataFrame(output)

In [74]:
Testing

,EmailOpen,EmailClickthrough,FormSubmit,WebVist,PageView,UserId
0,5,0,0,0,0,0000ecf58c94668
1,1,1,0,0,0,000132f21724ee2
2,3,0,0,0,0,0001b315b991cf0
3,4,0,0,0,0,000269150971d08
4,3,0,0,0,0,000297ba4fa5026
5,7,0,0,0,0,000342505b41079
6,1,0,0,0,0,0003b304dd7cfae
7,3,0,0,0,0,0003ee709c3b5dc
8,2,1,1,0,1,00049247bdf8ddf
9,1,1,1,0,1,0004cf13b5e07dd


In [63]:
final.shape

(265620, 7)

In [75]:
Testing['UserId'] = Testing.index 

In [76]:
Testing.index = range(132810)

In [77]:
output.index = range(132810)

In [78]:
final = pd.concat([Testing, output], axis=1)

In [79]:
final

,EmailOpen,EmailClickthrough,FormSubmit,WebVist,PageView,UserId,0,1
0,5,0,0,0,0,0,0.895500,0.104500
1,1,1,0,0,0,1,0.937414,0.062586
2,3,0,0,0,0,2,0.914077,0.085923
3,4,0,0,0,0,3,0.908662,0.091338
4,3,0,0,0,0,4,0.914077,0.085923
5,7,0,0,0,0,5,0.889740,0.110260
6,1,0,0,0,0,6,0.937414,0.062586
7,3,0,0,0,0,7,0.914077,0.085923
8,2,1,1,0,1,8,0.898805,0.101195
9,1,1,1,0,1,9,0.949747,0.050253


In [67]:
final.columns = ['EmailOpen','EmailClickthrough','FormSubmit','WebVist','PageView','UserId','No','Yes']

ValueError: Length mismatch: Expected axis has 7 elements, new values have 8 elements

In [68]:
Yes = 'Yes'

In [69]:
final2 = final.sort_values(by=Yes,ascending = False)

KeyError: 'Yes'

In [ ]:
Top1000 = final2.nlargest(1000, 'Yes')

In [ ]:
Top1000_Final = Top1000[['UserId','Yes']]

In [ ]:
Top1000_Final

In [ ]:
Top1000_Final.to_csv("Top_1000.csv")